### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Atrial_Fibrillation/GSE143924'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Define keys for sample characteristics dictionary
sample_char_dict_key_1 = 1  # Assuming 1 as it's the index containing 'Atrial_Fibrillation'

# Identify the row containing the 'Atrial_Fibrillation' information
if 'patient diagnosis: sinus rhythm after surgery' in clinical_data[sample_char_dict_key_1] or 'patient diagnosis: postoperative atrial fibrillation after surgery (POAF)' in clinical_data[sample_char_dict_key_1]:
    trait_row = sample_char_dict_key_1

# Dummy functions for age and gender variables as they are unavailable
def convert_age(value):
    return None

def convert_gender(value):
    return None

# Define the conversion function for 'Atrial_Fibrillation'
def convert_trait(value):
    try:
        value = value.split(':')[-1].strip()  # Extract the value after the colon and strip spaces
        if value == 'sinus rhythm after surgery':
            return 0
        elif value == 'postoperative atrial fibrillation after surgery (POAF)':
            return 1
        else:
            return None
    except Exception:
        return None

# Save metadata
save_cohort_info('GSE143924', './preprocessed/Atrial_Fibrillation/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Atrial_Fibrillation', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Atrial_Fibrillation/trait_data/GSE143924.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
